In [1]:
import matplotlib.font_manager as fm
myfont = fm.FontProperties(fname='./msyh.ttf')

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from talib.abstract import *
import seaborn as sbn
sbn.set()

In [34]:
mini_concept_index_pct_chg = pd.read_csv("mini_concept_index_pct_chg.csv").set_index(['tradeDate']).fillna(0)
mid_concept_index_pct_chg = pd.read_csv("mid_concept_index_pct_chg.csv").set_index(['tradeDate']).fillna(0)
big_concept_index_pct_chg = pd.read_csv("big_concept_index_pct_chg.csv").set_index(['tradeDate']).fillna(0)

In [ ]:
ax = mini_concept_index_pct_chg.plot(figsize=(21, 14))
legend = ax.legend()
for text in legend.texts:
    text.set_font_properties(myfont)

In [35]:
mini_concept_index_pct = mini_concept_index_pct_chg + 1.0
mid_concept_index_pct = mid_concept_index_pct_chg + 1.0
big_concept_index_pct = big_concept_index_pct_chg + 1.0

In [5]:
from influxdb import InfluxDBClient
from influxdb import SeriesHelper
host = '183.136.205.102'
port = 38086
user = 'root'
password = 'root'
dbname = 'chq'

myclient = InfluxDBClient(host, port, user, password, dbname)

In [62]:
id = 'test1'
class MySeriesHelper(SeriesHelper):
    # Meta class stores time series helper configuration.
    class Meta:
        # The client should be an instance of InfluxDBClient.
        client = myclient
        # The series name must be a string. Add dependent fields/tags in curly brackets.
        series_name = 'idx.' + str(id)
        # Defines all the fields in this time series.
        fields = ['close']
        # Defines all the tags for the series.
        tags = ['idx_name','dtype', 'counts']
        # Defines the number of data points to store prior to writing on the wire.
        bulk_size = 100
        # autocommit must be set to True when using bulk_size
        autocommit = True

In [63]:
def insert_to_db(data, dtype):
    for column in data.columns:
        for date in data.index.values:
            counts = concept_counts.loc[concept_counts['concept_name'] == column, 'counts'].values[0]
            close = data.loc[date, column]
            MySeriesHelper(time=date, idx_name=column, counts=counts, dtype=dtype, close=close)
    MySeriesHelper.commit()

In [64]:
concept = pd.read_csv("ths_concept.csv", dtype={'code': str})
concept.columns = ['concept_id', 'concept_name', 'stock_code', 'stock_name']
concept_counts = concept.groupby(['concept_id', 'concept_name']).size().reset_index(name='counts')

In [65]:
for data in [mini_concept_index_pct_chg, mid_concept_index_pct_chg, big_concept_index_pct_chg]:
    insert_to_db(data, 'pct_chg')

In [66]:
mini_concept_index_cumprod = mini_concept_index_pct.cumprod()
mid_concept_index_cumprod = mid_concept_index_pct.cumprod()
big_concept_index_cumprod = big_concept_index_pct.cumprod()

In [91]:
for data in [mini_concept_index_cumprod, mid_concept_index_cumprod, big_concept_index_cumprod]:
    insert_to_db(data, 'cumprod')

In [89]:
client = InfluxDBClient(host, port, user, password, dbname)
print("Drop database: " + dbname)
client.drop_database(dbname)


Drop database: chq


In [90]:
client = InfluxDBClient(host, port, user, password, dbname)
print("Create database: " + dbname)
client.create_database(dbname)

print("Create a retention policy")
client.create_retention_policy('awesome_policy', '200d', 3, default=True)

#     print("Switch user: " + dbuser)
#     client.switch_user(dbuser, dbuser_password)

#     print("Write points: {0}".format(json_body))
#     client.write_points(json_body)

#     print("Querying data: " + query)
#     result = client.query(query)

#     print("Result: {0}".format(result))

#     print("Switch user: " + user)
#     client.switch_user(user, password)

#     print("Drop database: " + dbname)
#     client.drop_database(dbname)


Create database: chq
Create a retention policy
